Import libraries

In [ ]:
import re
import string
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import MinMaxScaler 
from umap import UMAP # display in 3D? check stats text.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import datasets
# from haystack.
# from sklearn.feature_extraction.text import TfidfVectorizer

# pd.set_option

Choose dataset for EDA

In [ ]:
datasets.list_datasets()

In [ ]:
qa10topics = datasets.load_dataset('yahoo_answers_topics')

Check it out

In [ ]:
qa10topics

In [ ]:
qa10topics['train'].column_names

In [ ]:
qa10topics['train'].features

In [ ]:
labels = qa10topics['train'].features['topic'].names
labels

___

Let's look at the training set more closely

In [ ]:
qa10topics.set_format(type='pandas')
df = qa10topics['train'][:]

In [ ]:
df['topic'].value_counts() # balanced

In [ ]:
df.info() # no nulls in any column

In [ ]:
df.id.nunique() # id is unique id

In [ ]:
df.head(2)

___

Clean

In [ ]:
def cleaner(text):
    text = re.sub('<.{,5}>', ' ', text)
    # text = re.sub('[^a-zA-Z 0-9!,.;:]', ' ', text)
    text = re.sub(f'[^{string.printable}]', ' ', text)
    text = re.sub(' {2,}', ' ', text)
    text = text.lower().strip()
    return text

s = " hear that the mojave road is amazing!<br />\.. "
cleaner(s)

In [ ]:
for col in list(df.select_dtypes(include='object').columns):
    df[col] = df[col].apply(cleaner)

In [ ]:
df.head(2)

New column for readable topic labels

In [ ]:
labels # from earlier

In [ ]:
df['topic_name'] = df['topic'].apply(lambda x: labels[x])
df.head(2)

Out of curiosity, how do questions begin?

In [ ]:
df['q_start'] = df['question_title'].apply(lambda x: x.split()[0] if len(x.split())>0 else x)

In [ ]:
df.groupby('topic_name')['q_start'].value_counts().groupby('topic_name').nlargest(2)

# It seems that "who" is more common in Sports & Entertainment,
# while "why" is more common in Politics & Society.

In [ ]:
q_starts = df.groupby('topic_name')['q_start'].value_counts()
q_df = pd.DataFrame()
for idx in labels:
    q_df[idx] = q_starts.loc[idx].index[:10]
q_df

# Why do so many questions start with "I"?